### Sachs data

[Sachs Protein Data](https://perso.univ-rennes1.fr/valerie.monbet/GM/Sachs.html)

In [1]:
#pip install ../. 
#pip install cdt
from cdt.data import load_dataset
import networkx as nx
import jax
import jax.numpy as jnp
import jax.random as random
import numpy as np
import pandas as pd

print(f"JAX backend: {jax.default_backend()}")
key=random.PRNGKey(343)

def permute_random_rows(key, X, p=0.5):
    key, subk = random.split(key)
    permutation = random.permutation(subk, jnp.arange(X.shape[1]))
    
    key, subk = random.split(key)
    permute = random.bernoulli(subk, p=jnp.float32(p), shape=(X.shape[0],))

    [X := X.at[i,:].set(X[i,permutation]) for i in range(X.shape[0]) if permute[i]]
    
    return X, permute, permutation

def permute_directed_graph(graph, permutation):
    g, perm, perm_g = graph, permutation, jnp.zeros_like(graph)
    for i in range(g.shape[0]):
        for j in range(g.shape[1]):
                perm_g = perm_g.at[i,j].set(g[perm[i],perm[j]])
                
    return perm_g

No GPU automatically detected. Setting SETTINGS.GPU to 0, and SETTINGS.NJOBS to cpu_count.
No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)
I0000 00:00:1737034938.771853    1799 tfrt_cpu_pjrt_client.cc:349] TfrtCpuClient created.


JAX backend: cpu


In [2]:
# Load the graph and data
sachs_data, sachs_graph = load_dataset("sachs")

adjacency = jnp.array(nx.to_numpy_array(sachs_graph))
print('Adjacency matrix: \n', adjacency)

data = jnp.array(sachs_data.values)[:,:]
print('Data shape: \n', data.shape)

Adjacency matrix: 
 [[0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1. 0. 1. 1. 0. 0.]
 [1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1. 0. 1. 1. 1. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
Data shape: 
 (7466, 11)


In [3]:
# Process data
key, subk = random.split(key)
X, indicator, permutation = permute_random_rows(subk, data, p=0.5)
#X = jax.nn.standardize(X, axis=0)
print('Processed data shape: \n', X.shape)
print('Permutation: \n', permutation)

ground_truth_graphs = [adjacency, permute_directed_graph(adjacency, permutation)]
print('Ground truth graphs: \n', ground_truth_graphs)

Processed data shape: 
 (7466, 11)
Permutation: 
 [ 2  4  9  6  7  8 10  5  3  0  1]
Ground truth graphs: 
 [Array([[0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 1., 0., 1., 1., 0., 0.],
       [1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32), Array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 1., 0., 1., 1., 1., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0.

In [4]:
# Unpermuted data set
pd.DataFrame(X[~indicator,:]).describe()

,0,1,2,3,4,5,6,7,8,9,10
count,3706.000000,3706.000000,3706.000000,3706.000000,3706.000000,3706.000000,3706.000000,3706.000000,3706.000000,3706.000000,3706.000000
mean,126.226578,149.241562,51.623028,145.906372,27.600828,26.573629,79.875435,636.472717,30.360804,132.743011,72.062790
std,257.605286,391.166779,149.812042,266.308319,43.727592,50.692448,139.785034,672.737854,99.237152,511.181976,221.737915
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.010000,1.000000,1.000000,1.000000,1.000000
25%,30.799999,16.400000,9.390000,17.900000,9.470000,8.510000,23.299999,274.000000,4.180000,19.299999,7.840000
50%,53.799999,26.400000,16.400000,51.900002,17.799999,17.200001,37.200001,449.000000,12.600000,30.200001,18.100000
75%,103.000000,65.949999,26.900000,172.000000,33.700001,32.500000,72.300003,757.000000,23.450000,50.000000,50.000000
max,3820.000000,5829.000000,2267.000000,3619.000000,1084.000000,2571.000000,3555.000000,8896.000000,1611.000000,7499.000000,4740.000000


In [5]:
# Permuted data set
pd.DataFrame(X[indicator,:]).describe()

,0,1,2,3,4,5,6,7,8,9,10
count,3760.000000,3760.000000,3760.000000,3760.000000,3760.000000,3760.000000,3760.000000,3760.000000,3760.000000,3760.000000,3760.000000
mean,58.037865,26.477222,137.253372,82.440437,615.198242,30.322784,74.454910,26.687933,156.260208,121.948219,141.575821
std,194.643814,42.366215,478.097717,135.753296,615.223389,86.149490,209.520645,40.469086,328.633026,237.192398,362.621826
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,9.470000,9.650000,19.299999,23.299999,279.000000,4.730000,8.200000,8.510000,18.799999,31.100000,16.700001
50%,16.700001,17.600000,30.799999,37.200001,449.000000,13.050000,18.799999,17.299999,54.200001,53.799999,26.900000
75%,27.400000,31.900000,49.599998,72.300003,750.000000,23.500000,54.700001,31.900000,170.000000,103.000000,62.599998
max,6208.000000,1275.000000,6378.000000,1715.000000,8354.000000,1155.000000,3398.000000,1715.000000,9058.000000,4614.000000,7105.000000


### Create VaMSL model

In [6]:
from vamsl.models.graph import ErdosReniDAGDistribution, ScaleFreeDAGDistribution, UniformDAGDistributionRejection
from vamsl.models import MixtureLinearGaussian, MixtureDenseNonlinearGaussian
from vamsl.models import LinearGaussian, DenseNonlinearGaussian
from vamsl.target import make_graph_model

# BN settings
n_vars = X.shape[1] # number of variables in each component BN
struct_eq_type = 'linear' # BN function class: 'linear' or 'nonlinear'
graph_type = 'er' # Random graph structure: 'sf' (scale-free) or 'er' (Erdos-Renyi)

# Derived variables
n_components = len(jnp.unique(indicator))
linear = True if struct_eq_type == 'linear' else False

# Model specification 
graph_model = make_graph_model(n_vars=n_vars, graph_prior_str=graph_type, edges_per_node=2)
lik = MixtureLinearGaussian(n_vars=n_vars, obs_noise=0.1)  
component_lik = LinearGaussian(n_vars=n_vars, obs_noise=0.1)#, hidden_layers=(5,))

In [7]:
from vamsl.inference import VaMSL

# Create VaMSL and initialize posteriors
vamsl = VaMSL(x=X, graph_model=graph_model, mixture_likelihood_model=lik, component_likelihood_model=component_lik)
key, subk = random.split(key)
vamsl.initialize_posteriors(key=subk, n_components=n_components, n_particles=7, linear=linear)

print('Posterior shapes:')
posts = vamsl.get_posteriors()
print('q_z:     ' + str(posts[0].shape)) # [n_components, n_particls, d, l, 2]
print('q_theta: ' + str(posts[1].shape)) if linear else print('q_theta: ' + str(len(posts[1]))) # leading dim of n_components
print('log_q_c: ' + str(posts[2].shape)) # [n_observations, n_components]
print('q_pi:    ' + str(posts[3].shape)) # [n_components,]

Posterior shapes:
q_z:     (2, 7, 11, 11, 2)
q_theta: (2, 7, 11, 11)
log_q_c: (7466, 2)
q_pi:    (2,)


In [13]:
from sklearn.metrics import classification_report, confusion_matrix
# CAVI and SVGD vars
n_cavi_updates, steps = 0, 100

# CAVI-loop
for cavi_update in range(n_cavi_updates):
    key, subk = random.split(key)
    # Optimize q(Z, \Theta)
    vamsl.update_particle_posteriors(key=subk, steps=steps, callback_every=steps,
                                     callback=vamsl.visualize_callback(), linear=linear)

    # Update to optimal q(c) and q(\pi)
    vamsl.update_responsibilities_and_weights()
    print(f'CAVI update number {cavi_update+1}/{n_cavi_updates}')
    
    # Print current clustering
    order = vamsl.identify_MAP_classification_ordering(ground_truth_indicator=indicator)
    y_pred = [order[k] for k in [jnp.argmax(c_i) for c_i in vamsl.get_posteriors()[2]]]
    print('MAP clustering: \n', confusion_matrix(indicator, y_pred))
    
# Final CAVI update with more SVGD steps to ensure annealing unto acyclic graphs
key, subk = random.split(key)
#vamsl.update_particle_posteriors(key=subk, steps=1000, callback_every=200, callback=vamsl.visualize_callback(), linear=linear)
vamsl.update_responsibilities_and_weights()

In [14]:
# Compute optimal ordering with respect to MAP classification accuracy
order = vamsl.identify_MAP_classification_ordering(ground_truth_indicator=indicator)
print('Optimal order:')
print(order)

Optimal order:
[0 1]


In [15]:
from sklearn.metrics import classification_report, confusion_matrix

print('Sums of responsibilities:')
print(jnp.sum(jnp.exp(vamsl.get_posteriors()[2]), axis=0))
print('Sum of entropy of responsibilities:')
print(jnp.sum(jnp.exp(vamsl.get_posteriors()[2])*vamsl.get_posteriors()[2]))
y_true, order = indicator, vamsl.identify_MAP_classification_ordering(ground_truth_indicator=indicator)
y_pred = [order[k] for k in [jnp.argmax(c_i) for c_i in vamsl.get_posteriors()[2]]]
if n_components==2:
    print('Classification report:')
    print(classification_report(y_true=y_true, y_pred=y_pred, target_names=['Component 1', 'Component 2']))
confusion_matrix(y_true, y_pred)

Sums of responsibilities:
[3733. 3733.]
Sum of entropy of responsibilities:
0.0
Classification report:
              precision    recall  f1-score   support

 Component 1       0.99      1.00      0.99      3706
 Component 2       1.00      0.99      0.99      3760

    accuracy                           0.99      7466
   macro avg       0.99      0.99      0.99      7466
weighted avg       0.99      0.99      0.99      7466



array([[3700,    6],
       [  33, 3727]])

In [10]:
from sklearn.metrics import classification_report, confusion_matrix

print('Sums of responsibilities:')
print(jnp.sum(jnp.exp(vamsl.get_posteriors()[2]), axis=0))
print('Sum of entropy of responsibilities:')
print(jnp.sum(jnp.exp(vamsl.get_posteriors()[2])*vamsl.get_posteriors()[2]))
y_true, order = indicator, vamsl.identify_MAP_classification_ordering(ground_truth_indicator=indicator)
y_pred = [order[k] for k in [jnp.argmax(c_i) for c_i in vamsl.get_posteriors()[2]]]
if n_components==2:
    print('Classification report:')
    print(classification_report(y_true=y_true, y_pred=y_pred, target_names=['Component 1', 'Component 2']))
confusion_matrix(y_true, y_pred)

Sums of responsibilities:
[522. 478.]
Sum of entropy of responsibilities:
0.0
Classification report:
              precision    recall  f1-score   support

 Component 1       0.98      0.96      0.97       489
 Component 2       0.96      0.98      0.97       511

    accuracy                           0.97      1000
   macro avg       0.97      0.97      0.97      1000
weighted avg       0.97      0.97      0.97      1000



array([[470,  19],
       [  8, 503]])

In [16]:
from vamsl.metrics import expected_shd, threshold_metrics, neg_ave_log_likelihood

# Get component datasets
datas = [X[(indicator==k).flatten(),:] for k in range(n_components)]

# Loop over components and calculate metrics
for k, data, q_z_k, q_theta_k in zip(range(n_components), datas, vamsl.get_posteriors()[0], vamsl.get_posteriors()[1]):
    # Get particle distribution for component
    q_g_k = vamsl.particle_to_g_lim(q_z_k, vamsl.get_E()[order[k]])
    dist = vamsl.get_empirical(q_g_k, q_theta_k)
    
    # Calculate metrics
    eshd = expected_shd(dist=dist, g=ground_truth_graphs[order[k]])       
    auroc = threshold_metrics(dist=dist, g=ground_truth_graphs[order[k]])['roc_auc']
    #negll = neg_ave_log_likelihood(dist=dist, eltwise_log_likelihood=vamsl.eltwise_component_log_likelihood_observ, x=data.x_ho)
    
    print(f' Component {k+1:4d} |  E-SHD: {eshd:4.1f}    AUROC: {auroc:5.2f}')#    neg. LL {negll:5.2f}')

 Component    1 |  E-SHD: 29.0    AUROC:  0.55
 Component    2 |  E-SHD: 31.5    AUROC:  0.48


### Performance of DiBS

In [10]:
from vamsl.metrics import expected_shd, threshold_metrics, neg_ave_log_likelihood

# Get component datasets
datas = [X[(indicator==k).flatten(),:] for k in range(n_components)]

# Loop over components and calculate metrics
for k, data, q_z_k, q_theta_k in zip(range(n_components), datas, vamsl.get_posteriors()[0], vamsl.get_posteriors()[1]):
    # Get particle distribution for component
    q_g_k = vamsl.particle_to_g_lim(q_z_k, vamsl.get_E()[order[k]])
    dist = vamsl.get_empirical(q_g_k, q_theta_k)
    
    # Calculate metrics
    eshd = expected_shd(dist=dist, g=ground_truth_graphs[order[k]])       
    auroc = threshold_metrics(dist=dist, g=ground_truth_graphs[order[k]])['roc_auc']
    #negll = neg_ave_log_likelihood(dist=dist, eltwise_log_likelihood=vamsl.eltwise_component_log_likelihood_observ, x=data.x_ho)
    
    print(f' Component {k+1:4d} |  E-SHD: {eshd:4.1f}    AUROC: {auroc:5.2f}')#    neg. LL {negll:5.2f}')

 Component    1 |  E-SHD: 21.0    AUROC:  0.54
